In [ ]:
#!pip install pandas transformers torch

Special install of pytorch for apple sillicon

In [ ]:
#!pip3 install --pre torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/nightly/cpu

In [7]:
# Import necessary libraries
import pandas as pd
from transformers import pipeline
import utils

ModuleNotFoundError: No module named 'utils'

In [3]:
# Step 1: Load and Parse the CSV
# =================================

# Load the CSV file with categories and identifiers
categories = pd.read_csv('hierarchy/FoR_code_hierarchical_4_digit.csv')

# Load the CSV file with papers to be identified
papers = pd.read_csv('sample_data/dev_sample.csv')

# Display the DataFrame to understand its structure
categories.head()

,division,division_id,group,group_id
0,"Agricultural, Veterinary And Food Sciences",300000,Agricultural biotechnology,300100
1,"Agricultural, Veterinary And Food Sciences",300000,Agricultural biotechnology,300100
2,"Agricultural, Veterinary And Food Sciences",300000,Agricultural biotechnology,300100
3,"Agricultural, Veterinary And Food Sciences",300000,Agricultural biotechnology,300100
4,"Agricultural, Veterinary And Food Sciences",300000,Agricultural biotechnology,300100


In [4]:
# Step 2: Build the Hierarchical Structure with Identifiers
hierarchy_with_ids = build_hierarchy_with_ids(categories)
print(hierarchy_with_ids)

NameError: name 'build_hierarchy_with_ids' is not defined

In [19]:
# Step 3: Initialize the Hugging Face Model
# ==========================================

# Load the Hugging Face model for classification
classifier = pipeline("zero-shot-classification",
                      #model="facebook/bart-large-mnli")
                      model="MoritzLaurer/deberta-v3-large-zeroshot-v2.0",device = "mps")


def classify_text_with_ids(text, classifier, node, threshold=0.7):
    print(f"Classifying at level: {list(node.keys())}")

    # Classify text at the current node level with multi-label classification
    labels = list(node.keys())
    predictions = classifier(text, candidate_labels=labels, multi_label=True)

    # Display raw predictions for clarity
    print(f"Raw predictions: {dict(zip(predictions['labels'], predictions['scores']))}")

    # Normalize probabilities by the top prediction
    max_score = max(predictions['scores'])
    normalized_scores = [score / max_score for score in predictions['scores']]

    # Display normalized scores for clarity
    print(f"Normalized scores: {dict(zip(predictions['labels'], normalized_scores))}")

    # Filter candidates above the threshold
    valid_candidates = [
        (predictions['labels'][i], predictions['scores'][i])
        for i in range(len(labels))
        if normalized_scores[i] >= threshold
    ]

    print(f"Valid candidates above threshold {threshold}: {valid_candidates}")

    results = []
    for label, score in valid_candidates:
        child_node = node[label]
        print(f"Processing label: '{label}' with score: {score:.4f}")

        if child_node['children']:  # Reclassify at the lower level
            print(f"Descending into children of '{label}'")
            sub_results = classify_text_with_ids(text, classifier, child_node['children'], threshold)
            # Multiply probabilities (naive Bayes style)
            for sub_label, sub_prob, sub_path in sub_results:
                combined_prob = score * sub_prob
                combined_path = f"{label} -> {sub_label}"
                results.append((combined_path, combined_prob, [label] + sub_path))
                print(f"  Combined result: {combined_path} with probability {combined_prob:.4f}")
        else:
            results.append((label, score, [label]))
            print(f"Final result at leaf node: {label} with probability {score:.4f}")

    return results

In [22]:
def classify_text_with_ids(text, classifier, node, threshold=0.7, n_max=None):
    # Step 1: Classify text at the current node level with multi-label classification
    labels = list(node.keys())
    predictions = classifier(text, candidate_labels=labels, multi_label=True)

    # Normalize probabilities by the top prediction
    max_score = max(predictions['scores'])
    normalized_scores = [score / max_score for score in predictions['scores']]

    # Filter candidates above the threshold
    valid_candidates = [
        (predictions['labels'][i], predictions['scores'][i])
        for i in range(len(labels))
        if normalized_scores[i] >= threshold
    ]

    results = []
    for label, score in valid_candidates:
        child_node = node[label]
        if child_node['children']:  # Reclassify at the lower level
            sub_results = classify_text_with_ids(text, classifier, child_node['children'], threshold)
            # Multiply probabilities (naive Bayes style)
            for sub_label, sub_prob, sub_path in sub_results:
                results.append((f"{label} -> {sub_label}", score * sub_prob, [label] + sub_path))
        else:
            results.append((label, score, [label]))

    # Step 2: Final pass over all combined answers
    if results:
        # Find the maximum probability from the combined results
        max_combined_prob = max(result[1] for result in results)

        # Filter out results with probabilities below threshold * max_combined_prob
        filtered_results = [
            result for result in results
            if result[1] >= threshold * max_combined_prob
        ]

        # Sort the results by probability in descending order
        filtered_results.sort(key=lambda x: x[1], reverse=True)

        # Cut to the top n_max results if specified
        if n_max is not None:
            filtered_results = filtered_results[:n_max]

        results = filtered_results


    return results

In [23]:
#title = "From Maladaptation to Competition to Cooperation in the Evolution of Musical Behaviour"
#journal = "Musicae Scientiae"
#title = "Protruding masticatory (superfast) myosin heads from staggered thick filaments of dog jaw muscle revealed by X-ray diffraction"
#journal = "Journal of Biochemistry"
title = "Characterising infant and young child feeding practices and the consumption of poultry products in rural Tanzania: A mixed methods approach"
journal = "Maternal and Child Nutrition"
abstract = "Suboptimal breastfeeding practices, early initiation of complementary feeding, and monotonous cereal-based diets have been implicated as contributors to continuing high rates of child undernutrition in sub-Saharan Africa. Nutrition-sensitive interventions, including agricultural programs that increase access to nutrient-rich vegetables, legumes, and animal-source foods, have the potential to achieve sustainable improvements in children's diets. In the quest to evaluate the efficacy of such programs in improving growth and development in the first 2 years of life, there is a role for mixed methods research to better understand existing infant and young child feeding practices. This analysis forms part of a longitudinal study assessing the impact of improvements to poultry health and crop production on diets and growth of 503 randomly selected children from eight rural communities in Manyoni District in central Tanzania. Using an explanatory sequential design, the quantitative phase of data collection was conducted between May 2014 and May 2016, comprising six monthly structured questionnaires, four monthly household-level documentation of chicken and egg consumption, and fortnightly records of children's breastfeeding status. The subsequent qualitative phase involved in-depth interviews with a subset of 39 mothers in October 2016. Breastfeeding was almost universal (96.8%) and of long duration (mean = 21.7 months, SD = 3.6), but early initiation of complementary feeding was also common (74.4%; mean = 4.0 months, SD = 1.8), overwhelmingly driven by maternal perceptions of insufficient milk supply (95.0%). Chicken and eggs were infrequently eaten, but close associations between maternal and child consumption patterns (p <.001) suggest the potential for strategies that increase household-level consumption to bring nutritional benefits to young children."

def tst_fnc():
    #query = "Journal: "+journal+", Title: "+physics_paper
    #query = "Journal: "+journal+", Title: "+title
    #query = title
    #query = title + ". Abstract: "+ abstract
    #query = "Journal: "+journal+", Title: "+title+", Abstract: "+ abstract
    query = "Abstract: "+ abstract+", Title: "+title+", Journal: "+journal
 #   return classify_text_with_ids(query, classifier, hierarchy_with_ids,threshold=0.3,n_max=10)
    return classify_text_with_ids(query, classifier, hierarchy_with_ids,threshold=0.7)

#elapsed_time = timeit.timeit(tst_fnc, number=1)
#print(f"Elapsed time: {elapsed_time} seconds")
result = tst_fnc()
# Display the results
print("Final Classification Results:")
for path, prob, hierarchy_path in result:
    print(f"Path: {path}, Probability: {prob:.4f}\n")

Final Classification Results:
Path: Health Sciences -> Public health, Probability: 0.4437

Path: Health Sciences -> Epidemiology, Probability: 0.3378



In [24]:
result

[('Health Sciences -> Public health',
  0.4437040154943901,
  ['Health Sciences', 'Public health']),
 ('Health Sciences -> Epidemiology',
  0.3377682463194418,
  ['Health Sciences', 'Epidemiology'])]